## TikTok
To prawdodpodobnie jeden z najciekawszych portali do pobierania danych. Niestety oficjalne API służy raczej integracji filmów w innych aplikacjach, a nie pobieraniu danych. 

Powstało kilka płatnych pakietów, aby taką możliwość dostarczyć oraz 2 darmowe. 

Niestety są bardzi zawodne. Pierwszy z nich znajdziemy na stronie: 
https://github.com/davidteather/TikTok-Api

Paczkę instaluje się poprzez: 

In [ ]:
pip install TikTokApi

Pakiet korzysta z ukrytej przeglądarki, aby ją zainstalować potrzebna jest następująca komenda: 

In [ ]:
!python -m playwright install --with-deps

Do poprawnego funkcjonownaia pakietu potrzebna jest biblioteka obsługująca asymetryczne przetwarzanie zapytań. 

In [ ]:
pip install nest-asyncio

import nest_asyncio
nest_asyncio.apply()

### Przykłady i związane z nimi problemy 

Wykonałem dużo instalacji - mogę zaimporować bibliotekę: 

In [ ]:
from TikTokApi import TikTokApi
api = TikTokApi(use_test_endpoints=True)

Ściągamy profil dody - to powinno zadziałać: 

In [ ]:
doda_profil = api.user(username = "doda")
doda_profil.as_dict

Próbujemy zrealizować przykład z dokumetnacji dotyczący pobierania filmów w pętli:

https://github.com/davidteather/TikTok-Api/blob/master/examples/user_example.py

Uruchamia bariery obronne ze strony TikTok:

In [ ]:
for video in doda_profil.videos():
  print(video.id)
  video.as_dict
  print(video)

In [ ]:
for video in api.trending.videos():
  print(video.id)

Próbujemy realizować przykład z pobraniem filmu z dokumentacji - w domu mi nie wyszedł: 

https://github.com/davidteather/TikTok-Api/blob/master/examples/video_example.py 

In [ ]:
video = api.video(id="7041997751718137094")

#Bytes of the TikTok video
video_data = video.bytes()

with open("out.mp4", "wb") as out_file:
  out_file.write(video_data)

Jeżeli bardzo zależy nam na danych TikTok można spróbować drugiej paczki, ale nie gwarantuje sukcesu. 
https://github.com/avilash/TikTokAPI-Python

Na pewno, istnieje kilka rozwiązań płatnych, które obsługują takie działania:

https://stevesie.com/apps/tiktok-api

## Webscrapping - Beautiful Soup
Aby zrozumieć, co poszło nie tak warto samemu od podstaw prześledzić jak zbierane są informacje ze stron internetowych. 

Służy do tego kilka pakietów - zaczniemy od prostych stron bez większych interakcji z użytkownikiem.

Tu przyda nam się pakiet **Beautiful Soup**. 

Do ściągania przyda nam się kilka pakietów: 


In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

Ściągniemy aktualności dla portalu dotyczącego rynków kapitałowych:

https://psik.org.pl/pl/dane-i-aktualnosci/transakcje-czlonkow

Na początku zakodujemy w zminnej nazwę strony i numer z aktualnościami:

In [1]:
 webURL = "https://psik.org.pl/pl/dane-i-aktualnosci/transakcje-czlonkow?start=" + "10"

Na początku chcemy pobrać zawartość naszej strony. Tu warto wiedzieć o jednej sztuczce. Przeglądarki internetowe podczas połączeń podają serwerom tzw. *User-Agent string* - ciąg znaków, który mówi o przeglądarce i systemie operacyjnym. 

https://pl.wikipedia.org/wiki/User_agent

Informacje jak wyglądają takie łańcuchy znajdują się na stronach deweloperskich przeglądarek: 
*   Chrome: https://developer.chrome.com/docs/multidevice/user-agent/ 
*   Firefox: https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent

Zastosowanie takiego łańcucha jest często konieczne, aby otrzymać zawartość strony. Robi się to tak: 

In [4]:
# Słownik z UA String:
headers = {'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'}

# Pobieramy stronę
page = requests.get(webURL, headers = headers)

Kolejnym krokiem jest uruchomienie obiektu BS, który będzie czytał naszą stronę. 

In [5]:
soup = BeautifulSoup(page.content, 'html.parser', from_encoding="utf-8")

Cała praca z obiektem to w zasadzie powtarzanie dwóch elementów kodu. 

Jego określenie będzie wymagać spędzenia czasu z przeglądarką. 

In [10]:
# Klasa elementu strony, której szukamy: 
statName = "page-header"

# Polecenie find.all pobiera wszytskie elementy
temp = soup.find_all(class_= statName)

# Ta komenda pobierze tekst
tempList = [pt.get_text() for pt in temp]

# Drukujemy tekst
print(tempList)

['\n Transakcje Członków \n', '\n\n\t\t\t\t\t\t\t\t\tFundusz z Grupy MCI wychodzi z telemedycznego SiDLY\t\t\t\t\t\t\t\n', '\n\n\t\t\t\t\t\t\t\t\tAvallon MBO III większy o 44 mln EUR. Fundusz pozyskał łącznie 137,1 mln EUR na transakcje z menedżerami\t\t\t\t\t\t\t\n', '\n\n\t\t\t\t\t\t\t\t\tEnterprise Investors zainwestuje w rozwój Modular System\t\t\t\t\t\t\t\n', '\n\n\t\t\t\t\t\t\t\t\tAvallon MBO sprzedał spółkę Zaberd S.A.\t\t\t\t\t\t\t\n', '\n\n\t\t\t\t\t\t\t\t\tRP przejmuje Deeper, światowego lidera na rynku echosond dla wędkarzy\t\t\t\t\t\t\t\n', '\n\n\t\t\t\t\t\t\t\t\tMCI.TechVentures i MCI.CreditVentures sfinalizowały wyjście z PIGU – wiodącej platformy e-commerce w krajach bałtyckich.\t\t\t\t\t\t\t\n', '\n\n\t\t\t\t\t\t\t\t\tResource Partners inwestuje w Buglo, wiodącego producenta placów zabaw\t\t\t\t\t\t\t\n', '\n\n\t\t\t\t\t\t\t\t\tEnterprise Investors sprzedaje Wento\t\t\t\t\t\t\t\n', '\n\n\t\t\t\t\t\t\t\t\tResource Partners inwestuje w 7ANNA, twórców rowerowych marek prem

In [ ]:
# Drukowanie elementów - tabulatory
print(*tempList)

**Ćwiczenie** - zmień kod, aby wyświetlić teksty opisów, zamiast nagłówków

### BS - pobieranie linków

Możmey wykorzystać pakiet, aby znajdować dowolne elementy na stronie i ich atrybuty. Ten bloczek kodu pozwoli na zebranie wszystkich linków na stronie - zarówno wewnętrznych i wewnętrznych. 

Dzieki temu mogę np. zmapowac strony i poruszać się po kolejnych informacjach. 

In [13]:
for link in soup.findAll('a'):
    print(link.get('href'))

/
/en/research-and-news/market-information
/pl/
/pl/wsparcie-dla-ukrainy
/pl/
#zbiorki-finansowe
#zbiorki-rzeczowe
#doradztwo-prawne
#pomoc-medyczna
#pomoc-w-zatrudnieniu
#pozostale
#pozostale
/pl/o-private-equity/case-studies
/pl/o-private-equity/kodeks-standardow-profesjonalnych
/pl/o-private-equity/odpowiedzialne-inwestowanie
/pl/o-psik/stowarzyszenie
/pl/o-psik/zarzad
/pl/o-psik/statut-i-kodeks
/pl/o-psik/kontakt
/pl/czlonkostwo/fundusze
/pl/czlonkostwo/banki-doradcy-i-inwestorzy
/pl/czlonkostwo/jak-dolaczyc-do-psik
/pl/czlonkostwo/rejestracja
/pl/dane-i-aktualnosci/wiadomosci-psik
/pl/dane-i-aktualnosci/transakcje-czlonkow
/pl/dane-i-aktualnosci/statystyki
/pl/dane-i-aktualnosci/raporty
/pl/dane-i-aktualnosci/ksiazki
/pl/dane-i-aktualnosci/covid-19
/pl/dzialalnosc-spoleczna/komitet-ds-filantropii-przedsiebiorczej
/pl/dzialalnosc-spoleczna/program-psik-social-business-accelerator
/pl/dzialalnosc-spoleczna/fundacja-valores
/pl/wydarzenia
/pl/logowanie
https://psik.org.pl/pl/dane-i-a

## Strony z interakcjami - Selenium 

Beautiful Soup to ciekawe rozwiązanie dla prostych stron. Czasem jednak potrzebne są bardziej zaawansowane rozwiązania. W takim przypadku pomaga biblioteka Selenium. 

Fizycznie uruchamia ona przeglądarkę i pozwala na sterowanie nią za pomocą kodu. 

Do jej wdrożenia potrzebna jest instalacja sterownika. 
https://www.selenium.dev/documentation/webdriver/


Dla przeglądarki Chrome informacje znajdują się tutaj:
https://chromedriver.chromium.org/downloads


Wykonanie tych instalacji jest czasochłonne - zobaczymy przykłady bezpośrednio na moim komputerze. 

## Co dalej

Poczas kursu nauczyliśmy sie korzystać z różnych API. 

Mechanizmy te są uniwersalne - można je wykorzystać także do innych baz danych. 

Listing tego typu baz danych prowadzony jest na stronie: 
https://github.com/public-apis/public-apis#open-data

Zachęcam do eksplorowania w razie potrzeby. 